In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/buses/ahmedabad-bus-tickets"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def close_pop_up(driver):
    try:
        # Assume pop-up has a close button with a certain class or id
        pop_up_close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='close']"))
        )
        pop_up_close_button.click()
        time.sleep(1)  # Give it some time to close
    except Exception as e:
        # If no pop-up, or pop-up couldn't be closed, just pass
        pass


# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        close_pop_up(driver)
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 3):  # There are 2 pages
        try:
            load_page(driver, URL)
            
            if page >= 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('ahmedabad_bus_details.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


In [17]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Mahasagar Travels,A/C Sleeper (2+1),11:30,04h 30m,16:00,3.8,384,13 Seats available
1,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Rajwadi Travels,NON A/C Sleeper (2+1),12:00,04h 45m,16:45,3.5,INR 450,27 Seats available
2,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,AIRBUS EXPRESS SERVICE,Urbania A/C Seater (2+1),12:30,04h 30m,17:00,4.2,INR 510,9 Seats available
3,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Eagle Express,A/C Seater / Sleeper (2+1),12:30,05h 15m,17:45,3.1,INR 429,27 Seats available
4,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Manish Travels Private Limited,VE A/C Sleeper (2+1),13:00,05h 00m,18:00,3.8,370,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
947,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),21:10,07h 20m,04:30,1.5,INR 800,19 Seats available
948,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,VE A/C Sleeper (2+1),17:30,07h 30m,01:00,2.1,INR 723,34 Seats available
949,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),20:50,09h 25m,06:15,2.4,INR 800,22 Seats available
950,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,A/C Sleeper (2+1),23:00,07h 00m,06:00,0,INR 723,28 Seats available


In [15]:
import pandas as pd

# Load the first CSV file
df = pd.read_csv('ahmedabad_bus_details.csv')

# Write it to a new CSV file (or append it to an existing one)
df.to_csv('Ahmedabad_bus_details.csv', index=False)  # 'index=False' to avoid writing row numbers


FileNotFoundError: [Errno 2] No such file or directory: 'ahmedabad_bus_details.csv'

In [19]:
df = 

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Mahasagar Travels,A/C Sleeper (2+1),11:30,04h 30m,16:00,3.8,384,13 Seats available
1,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Rajwadi Travels,NON A/C Sleeper (2+1),12:00,04h 45m,16:45,3.5,INR 450,27 Seats available
2,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,AIRBUS EXPRESS SERVICE,Urbania A/C Seater (2+1),12:30,04h 30m,17:00,4.2,INR 510,9 Seats available
3,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Eagle Express,A/C Seater / Sleeper (2+1),12:30,05h 15m,17:45,3.1,INR 429,27 Seats available
4,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Manish Travels Private Limited,VE A/C Sleeper (2+1),13:00,05h 00m,18:00,3.8,370,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
947,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),21:10,07h 20m,04:30,1.5,INR 800,19 Seats available
948,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,VE A/C Sleeper (2+1),17:30,07h 30m,01:00,2.1,INR 723,34 Seats available
949,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),20:50,09h 25m,06:15,2.4,INR 800,22 Seats available
950,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,A/C Sleeper (2+1),23:00,07h 00m,06:00,0,INR 723,28 Seats available


In [23]:
df.to_csv('ahemedabad.csv', index=False)

In [27]:
pd.read_csv("ahemedabad.csv")

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Mahasagar Travels,A/C Sleeper (2+1),11:30,04h 30m,16:00,3.8,384,13 Seats available
1,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Rajwadi Travels,NON A/C Sleeper (2+1),12:00,04h 45m,16:45,3.5,INR 450,27 Seats available
2,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,AIRBUS EXPRESS SERVICE,Urbania A/C Seater (2+1),12:30,04h 30m,17:00,4.2,INR 510,9 Seats available
3,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Eagle Express,A/C Seater / Sleeper (2+1),12:30,05h 15m,17:45,3.1,INR 429,27 Seats available
4,Rajkot to Ahmedabad,https://www.redbus.in/bus-tickets/rajkot-to-ah...,Manish Travels Private Limited,VE A/C Sleeper (2+1),13:00,05h 00m,18:00,3.8,370,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
947,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),21:10,07h 20m,04:30,1.5,INR 800,19 Seats available
948,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,VE A/C Sleeper (2+1),17:30,07h 30m,01:00,2.1,INR 723,34 Seats available
949,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Shree Ram Travels (Modasa),A/C Sleeper (2+1),20:50,09h 25m,06:15,2.4,INR 800,22 Seats available
950,Ahmedabad to Bhuj,https://www.redbus.in/bus-tickets/ahmedabad-to...,Patel tours and travels,A/C Sleeper (2+1),23:00,07h 00m,06:00,0.0,INR 723,28 Seats available
